In [ ]:
pip install spacepy numpy pandas 

In [ ]:
from spacepy import pycdf
import numpy as np
import pandas as pd

cdf_path = "/work/AL1_ASW91_L2_BLK_20250630_UNP_9999_999999_V02.cdf"
cdf = pycdf.CDF(cdf_path)

epoch = cdf['epoch_for_cdf_mod'][...]
density = cdf['proton_density'][...]
speed = cdf['proton_bulk_speed'][...]
temperature = cdf['proton_thermal'][...]

FILLVAL = -1e31
density = np.where(density == FILLVAL, np.nan, density)
speed = np.where(speed == FILLVAL, np.nan, speed)
temperature = np.where(temperature == FILLVAL, np.nan, temperature)

if isinstance(epoch[0], pd.Timestamp) or isinstance(epoch[0], np.datetime64) or hasattr(epoch[0], 'year'):
    time_dt = pd.to_datetime(epoch)
else:
    try:
        time_dt = pycdf.epoch_to_datetime(epoch)
    except Exception as e:
        print("SpacePy epoch_to_datetime failed, using manual conversion:", e)
        time_dt = pd.to_datetime(epoch - 62167219200000, unit='ms', origin='1970-01-01')

df = pd.DataFrame({
    'time': time_dt,
    'density_cm3': density,
    'speed_km_s': speed,
    'temperature': temperature
})

df['density_ma'] = df['density_cm3'].rolling(window=60, min_periods=1).mean()
df['speed_ma'] = df['speed_km_s'].rolling(window=60, min_periods=1).mean()
df['temp_ma'] = df['temperature'].rolling(window=60, min_periods=1).mean()
df['density_grad'] = df['density_cm3'].diff().rolling(window=10, min_periods=1).mean()
df['speed_grad'] = df['speed_km_s'].diff().rolling(window=10, min_periods=1).mean()
df['combined_metric'] = df['density_cm3'] * df['speed_km_s']

df.to_csv('aditya_l1_swis_data.csv', index=False)
print("Data saved to 'aditya_l1_swis_data.csv'")

cdf.close()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e60a24d0-64ab-4d4c-924c-56a613793cdc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>